https://www.kaggle.com/datasets/redwankarimsony/heart-disease-data

In [1]:
import pandas as pd
df = pd.read_csv('raw.csv', index_col=0)
df

,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
id,,,,,,,,,,,,,,,
1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,58,Male,VA Long Beach,asymptomatic,NaN,385.0,True,lv hypertrophy,NaN,NaN,NaN,NaN,NaN,NaN,0
920,62,Male,VA Long Beach,atypical angina,120.0,254.0,False,lv hypertrophy,93.0,True,0.0,NaN,NaN,NaN,1


In [2]:
# capture columns with decimal values
cols_float = []

for col in df.select_dtypes(include='float64').columns:
    decimal_part = df[col].dropna().apply(lambda x: x - int(x))
    cond = (decimal_part > 0).sum() > 0
    if cond:
        cols_float.append(col)
        
cols_float

['oldpeak']

In [3]:
mapping = {
    0: 'No',
    1: 'Mild',
    2: 'Moderate',
    3: 'Severe',
    4: 'Critical',
}

In [4]:
df.num.replace(mapping, inplace=True)

In [5]:
df.num.value_counts()

num
No          411
Mild        265
Moderate    109
Severe      107
Critical     28
Name: count, dtype: int64

In [6]:
df.rename(columns={'num': 'disease'}, inplace=True)

In [7]:
df.dataset.value_counts()

dataset
Cleveland        304
Hungary          293
VA Long Beach    200
Switzerland      123
Name: count, dtype: int64

In [8]:
df.isna().sum().to_frame().T

,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,disease
0,0,0,0,0,59,30,90,2,55,55,62,309,611,486,0


In [9]:
df = df.drop(columns=['slope', 'ca', 'thal'])

In [10]:
from sklearn.experimental import enable_iterative_imputer

In [11]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [12]:
imputer = IterativeImputer()

In [13]:
mapping = {}

for column in df.select_dtypes(include=[object, bool]).columns:
    mapping[column] = dict(enumerate(df[column].astype('category').cat.categories))
    
mapping_rev = {column: {v: k for k, v in mapping[column].items()} for column in mapping}
mapping_rev

{'sex': {'Female': 0, 'Male': 1},
 'dataset': {'Cleveland': 0,
  'Hungary': 1,
  'Switzerland': 2,
  'VA Long Beach': 3},
 'cp': {'asymptomatic': 0,
  'atypical angina': 1,
  'non-anginal': 2,
  'typical angina': 3},
 'fbs': {False: 0, True: 1},
 'restecg': {'lv hypertrophy': 0, 'normal': 1, 'st-t abnormality': 2},
 'exang': {False: 0, True: 1},
 'disease': {'Critical': 0, 'Mild': 1, 'Moderate': 2, 'No': 3, 'Severe': 4}}

In [14]:
df.replace(mapping_rev, inplace=True)
df

,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,disease
id,,,,,,,,,,,,
1,63,1,0,3,145.0,233.0,1.0,0.0,150.0,0.0,2.3,3
2,67,1,0,0,160.0,286.0,0.0,0.0,108.0,1.0,1.5,2
...,...,...,...,...,...,...,...,...,...,...,...,...
919,58,1,3,0,NaN,385.0,1.0,0.0,NaN,NaN,NaN,3
920,62,1,3,1,120.0,254.0,0.0,0.0,93.0,1.0,0.0,1


In [15]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import LabelEncoder

# Impute missing values
imputer = IterativeImputer(random_state=0)
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

In [16]:
df_imputed

,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,disease
0,63.0,1.0,0.0,3.0,145.000000,233.0,1.0,0.0,150.000000,0.000000,2.300000,3.0
1,67.0,1.0,0.0,0.0,160.000000,286.0,0.0,0.0,108.000000,1.000000,1.500000,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
918,58.0,1.0,3.0,0.0,139.251196,385.0,1.0,0.0,124.432518,0.699513,1.287899,3.0
919,62.0,1.0,3.0,1.0,120.000000,254.0,0.0,0.0,93.000000,1.000000,0.000000,1.0


In [17]:
dff = df_imputed.copy()

for column, mv in mapping.items():
    dff[column] = dff[column].round(0).replace(mv)

In [18]:
df_imputed = dff.convert_dtypes()

# convert to int if boolean
dtypes = df_imputed.dtypes
mapping = {column: 'int' for column in dtypes[dtypes == 'boolean'].index}
df_imputed = df_imputed.astype(mapping)
df_imputed

,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,disease
0,63,Male,Cleveland,typical angina,145.0,233.0,1,lv hypertrophy,150.0,0,2.3,No
1,67,Male,Cleveland,asymptomatic,160.0,286.0,0,lv hypertrophy,108.0,1,1.5,Moderate
...,...,...,...,...,...,...,...,...,...,...,...,...
918,58,Male,VA Long Beach,asymptomatic,139.251196,385.0,1,lv hypertrophy,124.432518,1,1.287899,No
919,62,Male,VA Long Beach,atypical angina,120.0,254.0,0,lv hypertrophy,93.0,1,0.0,Mild


In [19]:
for col in df_imputed.select_dtypes(include='float64').columns:
    if col not in cols_float:
        df_imputed[col] = df_imputed[col].round(0).astype(int)

In [20]:
df_imputed

,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,disease
0,63,Male,Cleveland,typical angina,145,233,1,lv hypertrophy,150,0,2.3,No
1,67,Male,Cleveland,asymptomatic,160,286,0,lv hypertrophy,108,1,1.5,Moderate
...,...,...,...,...,...,...,...,...,...,...,...,...
918,58,Male,VA Long Beach,asymptomatic,139,385,1,lv hypertrophy,124,1,1.287899,No
919,62,Male,VA Long Beach,atypical angina,120,254,0,lv hypertrophy,93,1,0.0,Mild


In [21]:
df_imputed.to_csv('output/filled_missing.csv')

In [62]:
df_imputed.isna().sum().to_frame().T

,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,num
0,0,0,0,0,0,0,0,0,0,0,0,0


In [63]:
from sklearn.impute import KNNImputer

In [64]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [8]:
columns = df.dtypes == 'object'

features_categorical = columns[columns].index.tolist()
features_numeric = columns[~columns].index.tolist()

In [9]:
features_categorical

['sex', 'dataset', 'cp', 'fbs', 'restecg', 'exang']

1. Aprenden a transformar las variables categoricas
2. Aprender a hacer el encoding

In [10]:
encoder = OneHotEncoder(handle_unknown='ignore')

transformer = ColumnTransformer(transformers=[
    ('encoder', encoder, features_categorical),
], remainder='passthrough')

In [11]:
imputer = KNNImputer(n_neighbors=2)

In [12]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline(steps=[
    ('transformer', transformer),
    ('imputer', imputer),
])

In [19]:
dff = pipeline.fit_transform(df)
dff = pd.DataFrame(dff)
dff.isna().sum().to_frame().T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
dff.columns = transformer.get_feature_names_out()

In [21]:
dff

,encoder__sex_Female,encoder__sex_Male,encoder__dataset_Cleveland,encoder__dataset_Hungary,encoder__dataset_Switzerland,encoder__dataset_VA Long Beach,encoder__cp_asymptomatic,encoder__cp_atypical angina,encoder__cp_non-anginal,encoder__cp_typical angina,encoder__fbs_False,encoder__fbs_True,encoder__fbs_nan,encoder__restecg_lv hypertrophy,encoder__restecg_normal,encoder__restecg_st-t abnormality,encoder__restecg_nan,encoder__exang_False,encoder__exang_True,encoder__exang_nan,remainder__age,remainder__trestbps,remainder__chol,remainder__thalch,remainder__oldpeak,remainder__num
0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,63.0,145.0,233.0,150.0,2.3,0.0
1,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,67.0,160.0,286.0,108.0,1.5,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
918,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,58.0,132.0,385.0,128.5,0.5,0.0
919,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,62.0,120.0,254.0,93.0,0.0,1.0


In [ ]:
column

'age'

In [97]:
features = transformer.get_feature_names_out()

l = {}
for i in features:
    column = i.split('__')[1]
    try:
        column,value = column.split('_')
        if column not in l:
            l[column] = [value]
        else:
            l[column].append(value)
    except:
        pass
    
l

AttributeError: 'ColumnTransformer' object has no attribute 'transformers_'

In [89]:
df = pd.DataFrame(dff, columns=features)

In [91]:
df.columns = [i.split('__')[1] for i in df.columns]
df

,sex_Female,sex_Male,dataset_Cleveland,dataset_Hungary,dataset_Switzerland,dataset_VA Long Beach,cp_asymptomatic,cp_atypical angina,cp_non-anginal,cp_typical angina,fbs_False,fbs_True,fbs_nan,restecg_lv hypertrophy,restecg_normal,restecg_st-t abnormality,restecg_nan,exang_False,exang_True,exang_nan,age,trestbps,chol,thalch,oldpeak,num
0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,63.0,145.0,233.0,150.0,2.3,0.0
1,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,67.0,160.0,286.0,108.0,1.5,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
918,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,58.0,132.0,385.0,128.5,0.5,0.0
919,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,62.0,120.0,254.0,93.0,0.0,1.0


In [92]:
df

,sex_Female,sex_Male,dataset_Cleveland,dataset_Hungary,dataset_Switzerland,dataset_VA Long Beach,cp_asymptomatic,cp_atypical angina,cp_non-anginal,cp_typical angina,fbs_False,fbs_True,fbs_nan,restecg_lv hypertrophy,restecg_normal,restecg_st-t abnormality,restecg_nan,exang_False,exang_True,exang_nan,age,trestbps,chol,thalch,oldpeak,num
0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,63.0,145.0,233.0,150.0,2.3,0.0
1,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,67.0,160.0,286.0,108.0,1.5,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
918,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,58.0,132.0,385.0,128.5,0.5,0.0
919,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,62.0,120.0,254.0,93.0,0.0,1.0


In [15]:
imputer.fit(df)

ValueError: could not convert string to float: 'Male'

In [12]:
df.dropna()

,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,num
id,,,,,,,,,,,,
1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,0
2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,2
...,...,...,...,...,...,...,...,...,...,...,...,...
918,55,Male,VA Long Beach,asymptomatic,122.0,223.0,True,st-t abnormality,100.0,False,0.0,2
920,62,Male,VA Long Beach,atypical angina,120.0,254.0,False,lv hypertrophy,93.0,True,0.0,1


In [ ]:
df